# Topic Modeling with Gensim

*Notebook version: 1.2402.0701*

We’re going to use the gensim implementations because they offer more functionality out of the box

## Library

In [1]:
!pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.3 MB/s eta 0:00:00


In [2]:
import nltk
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string

from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

<br>
<br>

## Helper Function

In [3]:
def tokenize_clean(text):
  return preprocess_string(text)

In [4]:
stopwords = nltk.corpus.stopwords.words('indonesian')
def remove_stopwords(tokenized_text):

    cleaned_token = []
    for token in tokenized_text:
        if token not in stopwords:
            cleaned_token.append(token)

    return cleaned_token

In [5]:
def stemming_text(tokenized_text):

    #stem using Sastrawi StemmerFactory
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    stems = []
    for token in tokenized_text:
        stems.append(stemmer.stem(token))

    return stems

In [6]:
def text_preprocessing(text):

    prep01 = tokenize_clean(text)
    prep02 = remove_stopwords(prep01)
    prep03 = stemming_text(prep02)

    return prep03

<br>
<br>

## Read Dataset

In [7]:
!mkdir -p dataset
!wget https://raw.githubusercontent.com/project303/dataset/master/Berita.txt -P dataset
!wget https://raw.githubusercontent.com/project303/dataset/master/Judul-Berita.txt -P dataset

--2024-02-06 16:57:30--  https://raw.githubusercontent.com/project303/dataset/master/Berita.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76212 (74K) [text/plain]
Saving to: ‘dataset/Berita.txt’

Berita.txt          100%[===================>]  74.43K  --.-KB/s    in 0.01s   

2024-02-06 16:57:30 (5.82 MB/s) - ‘dataset/Berita.txt’ saved [76212/76212]

--2024-02-06 16:57:30--  https://raw.githubusercontent.com/project303/dataset/master/Judul-Berita.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1751 (1.7K) [text/plain]
Sa

In [25]:
# read article title
article_titles = open('dataset/Judul-Berita.txt').read().split('\n')
len(article_titles)

31

In [8]:
# read article content
article = open('dataset/Berita.txt', encoding="utf8").read().split('BERHENTI DISINI')
len(article)

31

In [9]:
article[0]

'<html lang="en">\n\n    <head>\n    <title>Nasabah Bank Mandiri Mengaku Uangnya Raib Rp49 Juta</title>\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="robots" content="index, follow" />\n\n<body>\n<p>Jakarta, CNN Indonesia -- Agung Rahman, seorang nasabah Bank Mandiri di Karawang mengaku tiba-tiba kehilangan uang di rekening banknya. Akibat kejadian tersebut dia mengalami kerugian hampir Rp49 juta.</p>\n\n<p>Agung mengatakan sebelum kejadian tersebut saldo tabungan yang telah dihimpunnya selama setahun tersebut mencapai Rp49.667.000. Pada 10 Juni lalu, dia terakhir kali mengecek tabungan tersebut.</p>\n\n<p>Namun, sontak ia kaget. Pasalnya, sisa saldo di tabungannya tinggal Rp832.324.</p>\n\n<p>"Jadi, tiba- tiba hilang," katanya seperti dikutip dari Antara, Selasa (12/6).</p>\n\n<p>Agung kaget karena saldo tersebut berkurang banyak tanpa ia melakukan transaksi sama sekali. Ia mengklaim sudah berupaya mengecek ke Bank Mandiri.\n\nDari hasil p

## Preprocessing

In [10]:
# remove HTML tag
article_clean = []
for text in article:
    text = BeautifulSoup(text, 'html.parser').getText()
    article_clean.append(text)
article = article_clean

In [11]:
article[0]

'\n\nNasabah Bank Mandiri Mengaku Uangnya Raib Rp49 Juta\n\n\n\n\nJakarta, CNN Indonesia -- Agung Rahman, seorang nasabah Bank Mandiri di Karawang mengaku tiba-tiba kehilangan uang di rekening banknya. Akibat kejadian tersebut dia mengalami kerugian hampir Rp49 juta.\nAgung mengatakan sebelum kejadian tersebut saldo tabungan yang telah dihimpunnya selama setahun tersebut mencapai Rp49.667.000. Pada 10 Juni lalu, dia terakhir kali mengecek tabungan tersebut.\nNamun, sontak ia kaget. Pasalnya, sisa saldo di tabungannya tinggal Rp832.324.\n"Jadi, tiba- tiba hilang," katanya seperti dikutip dari Antara, Selasa (12/6).\nAgung kaget karena saldo tersebut berkurang banyak tanpa ia melakukan transaksi sama sekali. Ia mengklaim sudah berupaya mengecek ke Bank Mandiri.\n\nDari hasil pengecekan tersebut Bank Mandiri menyatakan bahwa berkurangnya saldo tabungan Agung terjadi akibat transaksi di rekeningnya.\n\n"Tapi ini janggal, karena saya merasa tidak pernah melakukan transaksi selama ini," kata

In [12]:
print(article[0])



Nasabah Bank Mandiri Mengaku Uangnya Raib Rp49 Juta




Jakarta, CNN Indonesia -- Agung Rahman, seorang nasabah Bank Mandiri di Karawang mengaku tiba-tiba kehilangan uang di rekening banknya. Akibat kejadian tersebut dia mengalami kerugian hampir Rp49 juta.
Agung mengatakan sebelum kejadian tersebut saldo tabungan yang telah dihimpunnya selama setahun tersebut mencapai Rp49.667.000. Pada 10 Juni lalu, dia terakhir kali mengecek tabungan tersebut.
Namun, sontak ia kaget. Pasalnya, sisa saldo di tabungannya tinggal Rp832.324.
"Jadi, tiba- tiba hilang," katanya seperti dikutip dari Antara, Selasa (12/6).
Agung kaget karena saldo tersebut berkurang banyak tanpa ia melakukan transaksi sama sekali. Ia mengklaim sudah berupaya mengecek ke Bank Mandiri.

Dari hasil pengecekan tersebut Bank Mandiri menyatakan bahwa berkurangnya saldo tabungan Agung terjadi akibat transaksi di rekeningnya.

"Tapi ini janggal, karena saya merasa tidak pernah melakukan transaksi selama ini," katanya.

Atas kejad

In [13]:
# proses ini memerlukan waktu sekitar 3 menit
tokenized_data = []
for text in article:
    tokenized_data.append(text_preprocessing(text))

In [14]:
len(tokenized_data)

31

In [15]:
print(tokenized_data[0])

['nasabah', 'bank', 'mandiri', 'aku', 'uang', 'raib', 'juta', 'jakarta', 'cnn', 'indonesia', 'agung', 'rahman', 'nasabah', 'bank', 'mandiri', 'karawang', 'aku', 'hilang', 'uang', 'reken', 'bank', 'akibat', 'jadi', 'alami', 'rugi', 'juta', 'agung', 'jadi', 'saldo', 'tabung', 'himpun', 'tahun', 'capai', 'juni', 'kali', 'ecek', 'tabung', 'sontak', 'kaget', 'pasal', 'sisa', 'saldo', 'tabung', 'tinggal', 'hilang', 'kutip', 'selasa', 'agung', 'kaget', 'saldo', 'kurang', 'transaksi', 'klaim', 'upaya', 'ecek', 'bank', 'mandiri', 'hasil', 'kece', 'bank', 'mandiri', 'kurang', 'saldo', 'tabung', 'agung', 'akibat', 'transaksi', 'rekening', 'janggal', 'transaksi', 'ata', 'jadi', 'janggal', 'agung', 'putus', 'lapor', 'polr', 'karawang', 'kasu', 'tangan', 'polisi', 'konfirmasi', 'sekretari', 'usaha', 'bank', 'mandiri', 'roh', 'hafa', 'aku', 'haru', 'ecek', 'kasu', 'telusur', 'nasabah', 'khawatir', 'nasabah', 'transaksi', 'dana', 'kembali', 'tega', 'roh', 'cnnindonesia', 'com', 'telusur', 'sambung', '

In [16]:
len(tokenized_data[0])

120

<br>
<br>

## Create The Model

In [17]:
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)

# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]

In [20]:
NUM_TOPICS = 3

# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha = 'auto', eval_every=5)#, per_word_topics=True)

# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [21]:
print("LDA Model:")

for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))

print("=" * 20)

print("LSI Model:")

for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))

print("=" * 20)

LDA Model:
Topic #0: 0.007*"main" + 0.007*"persen" + 0.007*"balap" + 0.006*"indonesia" + 0.006*"diskon" + 0.006*"jakarta" + 0.006*"lawan" + 0.006*"dunia" + 0.006*"tanding" + 0.005*"vettel"
Topic #1: 0.029*"persen" + 0.007*"indonesia" + 0.007*"lemah" + 0.006*"dolar" + 0.006*"jakarta" + 0.006*"mu" + 0.005*"uang" + 0.005*"belanja" + 0.005*"bunga" + 0.005*"dunia"
Topic #2: 0.007*"persen" + 0.006*"jakarta" + 0.006*"indonesia" + 0.005*"duga" + 0.005*"tabung" + 0.005*"polri" + 0.005*"cnn" + 0.005*"uang" + 0.004*"polisi" + 0.004*"kasu"
LSI Model:
Topic #0: -0.744*"persen" + -0.218*"lemah" + -0.172*"dolar" + -0.164*"mu" + -0.135*"indek" + -0.129*"bunga" + -0.116*"kuat" + -0.112*"uang" + -0.104*"dagang" + -0.101*"indonesia"
Topic #1: 0.401*"novel" + 0.324*"oknum" + 0.315*"duga" + 0.282*"jender" + 0.178*"polisi" + 0.171*"kpk" + 0.147*"teror" + 0.145*"kera" + 0.143*"kasu" + 0.137*"air"
Topic #2: -0.442*"main" + -0.262*"dunia" + -0.251*"argentina" + -0.250*"tanding" + -0.238*"lawan" + -0.202*"piala

<br>
<br>

## Test The Model

In [22]:
print("article[0]")
print("LDA Model:")
print(lda_model[corpus[0]])

print("")
print("LSA Model:")
print(lsi_model[corpus[0]])

article[0]
LDA Model:
[(2, 0.98689944)]

LSA Model:
[(0, -1.6138115051806685), (1, 1.8434785272885046), (2, -0.2963407290001005)]


In [24]:
print(tokenized_data[0])

['nasabah', 'bank', 'mandiri', 'aku', 'uang', 'raib', 'juta', 'jakarta', 'cnn', 'indonesia', 'agung', 'rahman', 'nasabah', 'bank', 'mandiri', 'karawang', 'aku', 'hilang', 'uang', 'reken', 'bank', 'akibat', 'jadi', 'alami', 'rugi', 'juta', 'agung', 'jadi', 'saldo', 'tabung', 'himpun', 'tahun', 'capai', 'juni', 'kali', 'ecek', 'tabung', 'sontak', 'kaget', 'pasal', 'sisa', 'saldo', 'tabung', 'tinggal', 'hilang', 'kutip', 'selasa', 'agung', 'kaget', 'saldo', 'kurang', 'transaksi', 'klaim', 'upaya', 'ecek', 'bank', 'mandiri', 'hasil', 'kece', 'bank', 'mandiri', 'kurang', 'saldo', 'tabung', 'agung', 'akibat', 'transaksi', 'rekening', 'janggal', 'transaksi', 'ata', 'jadi', 'janggal', 'agung', 'putus', 'lapor', 'polr', 'karawang', 'kasu', 'tangan', 'polisi', 'konfirmasi', 'sekretari', 'usaha', 'bank', 'mandiri', 'roh', 'hafa', 'aku', 'haru', 'ecek', 'kasu', 'telusur', 'nasabah', 'khawatir', 'nasabah', 'transaksi', 'dana', 'kembali', 'tega', 'roh', 'cnnindonesia', 'com', 'telusur', 'sambung', '

In [26]:
article_titles[0]

'Nasabah Bank Mandiri Mengaku Uangnya Raib Rp49 Juta'

In [28]:
text = "Pertandingan berjalan dengan seru. Tim lawan berhasil dikalahkan dengan skor 1-0."
bow = dictionary.doc2bow(text_preprocessing(text))

print(lda_model[bow])
print(lsi_model[bow])
print(bow)

[(0, 0.83011204), (1, 0.09617264), (2, 0.073715255)]
[(0, -0.12598201478273724), (1, 0.2771017671407879), (2, -0.7973742999991417)]
[(18, 1), (112, 1), (354, 1), (401, 1), (411, 1), (448, 1), (838, 1)]


In [31]:
len(dictionary)

1740

In [37]:
from gensim import similarities

lda_index = similarities.MatrixSimilarity(lda_model[corpus])

# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])

# Top most similar documents:
print(similarities[:10])

# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(article[document_id][:1000])

[(7, 0.9908834), (4, 0.98951226), (9, 0.98951226), (10, 0.98951226), (13, 0.98951226), (15, 0.98951226), (20, 0.98951226), (22, 0.98951226), (27, 0.98951226), (28, 0.98951226)]



Jakarta, CNN Indonesia -- Menteri Sosial Idrus Marham menyatakan bahwa pemerintah akan menaikkan bantuan Program Keluarga Harapan (PKH) pada 2019 mendatang. Jika tahun ini bantuan hanya sebesar Rp1,89 juta per keluarga per tahun, pada tahun depan bantuan akan dinaikkan menjadi Rp2 juta per keluarga per tahun.

Bahkan, kenaikan bisa maksimal sampai dengan Rp3,5 juta per keluarga per tahun.

"Tidak hanya dalam bentuk PKH saja, akan ada bantuan sosial dan subsidi juga dari kementerian lain untuk masyarakat agar pengentasan kemiskinan bisa cepat dilakukan," katanya dalam pernyataan yang dikeluarkan Minggu (25/6).

Menteri Perdagangan Enggartiasto Lukita mengatakan selain bantuan PKH untuk membantu keluarga miskin keluar dari kemiskinan, pemerintah juga akan memberikan bantuan tenda atau gerobak.

Dengan bantuan t

<br>
<br>

## Visualization

In [38]:
# Install pyLDAVis (specific version for Google Collab)
!pip install pyLDAvis==2.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97718 sha256=91dfadf108a240cfcade986c17b10e497781cc5387b58cf2816aa3f5eea3d681
  Stored in directory: /root/.cache/pip/wheels/d9/93/d6/16c95da19c32f037fd75135ea152d0df37254c25cd1a8b4b6c
Successfully built pyLDAvis


In [39]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
panel

/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.037846 -0.014488       1        1  39.208080
1      0.037966 -0.014351       2        1  34.498084
2     -0.000120  0.028839       3        1  26.293836, topic_info=         Term       Freq      Total Category  logprob  loglift
152    persen  87.000000  87.000000  Default  30.0000  30.0000
1285    balap  18.000000  18.000000  Default  29.0000  29.0000
62     tabung  11.000000  11.000000  Default  28.0000  28.0000
1329   vettel  14.000000  14.000000  Default  27.0000  27.0000
1670  belanja  14.000000  14.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
688      aman   4.376853  14.079410   Topic3  -5.8694   0.1675
108     indek   4.409658  16.338559   Topic3  -5.8619   0.0261
18      hasil   4.478894  20.325813   Topic3  -5.8463  -0.1767
427     piala   4.246732  17.812618   Topic3  -5.8995  -0.0979
91      dolar   4.278281  21.327817   Topic3  -5.8921  -0.2706

[215 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
72        1  0.204114   acu
72        2  0.714399   acu
72        3  0.102057   acu
1463      1  0.320070   aji
1463      2  0.320070   aji
...     ...       ...   ...
1214      2  0.120550  voni
1214      3  0.482200  voni
1260      1  0.131347  wood
1260      2  0.656736  wood
1260      3  0.262694  wood

[438 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

<br>
<br>

## How Dictionary and doc2bow Work

In [32]:
texts = [['durian', 'belimbing', 'cempedak' ], ['apel', 'belimbing']]

dct = corpora.Dictionary(texts)  # initialize a Dictionary

In [33]:
len(dct)

4

In [34]:
dct.keys()

[0, 1, 2, 3]

In [35]:
dct[1]

'cempedak'

In [36]:
dct.doc2bow(["belimbing", "apel", "non_existent_word"])

[(0, 1), (3, 1)]

<br>
<br>

## Revision History


Release: 1.2102.0601
*   First release

Release: 1.2402.0701
*   Change preprocessing process
*   Add how Dictionary dan doc2bow works